In [2]:
import os
import pandas as pd
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse

In [ ]:
df_userforgenre = pd.read_csv('./data/user_for_genre.csv')

In [ ]:
df_userforgenre

In [ ]:
app = FastAPI()

In [ ]:
def get_user_for_genre(genero: str = None )-> dict:
    """reune el jugador que mas ah jugado por genero especifico    
    Argumentos:
        genero (str, optional): insertar genero del juego. Defaults to None.
    Returns:
        dict:
    """
    df_filtrado_por_genero = df_userforgenre[df_userforgenre['genres'] == genero]
    usuario_horas = df_filtrado_por_genero.groupby('user_id')['playtime_forever'].sum()
    max_tiempo_x_usuario = usuario_horas.idxmax()

    df_max_usuario = df_filtrado_por_genero[df_filtrado_por_genero['user_id'] == max_tiempo_x_usuario]
    horas_x_anio = df_max_usuario.groupby('release_year')['playtime_forever'].sum().reset_index()
    lista_x_anio = [
        {'Año':year ,  'Horas': hours} for year, hours in zip(horas_x_anio['release_year'], horas_x_anio['playtime_forever'])
    ]

    resultado = {
        'Usuario con mas horas jugadas para el genero {}'.format(genero) : max_tiempo_x_usuario,
        'horas jugadas ': lista_x_anio
    }

    return resultado

In [ ]:
get_user_for_genre('free to play')

In [ ]:
df_steam_games = pd.read_csv('./data/df_steam_games.csv')
df_users_items = pd.read_csv('./data/users_items.csv')

In [ ]:
def playtime_genre(genero: str):

    try:
        if f'genres_{genero}_new' not in df_steam_games.columns:
            raise HTTPException(status_code=404, detail=f"No hay datos para el género {genero}")

        genre_data = df_steam_games[df_steam_games[f'genres_{genero}_new'] == 1]
        genre_data['user_id'] = genre_data['user_id'].astype('object')

        merged_data = pd.merge(genre_data, df_users_items, on='user_id', how='inner')
        year_with_most_playtime = merged_data.groupby('año de lanzamiento')['playtime_forever'].sum().idxmax()

        return {"Año de lanzamiento con más horas jugadas para " + genero: int(year_with_most_playtime)}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error interno: {str(e)}")

In [ ]:
playtime_genre('action')

In [3]:
from pydantic import BaseModel
from dateutil import parser
from typing import List
import pyarrow.parquet as pq
import os
import string
import re

In [ ]:
def developer(desarrollador):
    '''
    Esta función devuelve información sobre una empresa desarrolladora de videojuegos.
         
    Args:
        desarrollador (str): Nombre del desarrollador de videojuegos.
    
    Returns:
        dict: Un diccionario que contiene información sobre la empresa desarrolladora.
            - 'cantidad_por_año' (dict): Cantidad de items desarrollados por año.
            - 'porcentaje_gratis_por_año' (dict): Porcentaje de contenido gratuito por año según la empresa desarrolladora.
    '''
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict